In [2]:
import pandas as pd             
import matplotlib.pyplot as plt 
import datetime as dt
import numpy as np

# these are new 
import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import shutil                   # file management tools 
import os                       # operating system tools (check files)

from census import Census # This is new...

import geopandas as gpd # this is the main geopandas 
from shapely.geometry import Point, Polygon # also needed

import pyarrow as pa
import pyarrow.parquet as pq


from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

## Interactive Covid-19 map by state  

---

### Step \#1. Grab data from https://github.com/camckenzie/COVID-19-Interactive-Map



In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/camckenzie/COVID-19-Interactive-Map/main/covid19_cases.csv",skiprows=3,header = None)

In [4]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,State/Territory,Total Cases,Confirmed Cases,Probable Cases,Cases in Last 7 Days,Case Rate per 100000,Total Deaths,Confirmed Deaths,Probable Deaths,Deaths in Last 7 Days,Death Rate per 100000,7-Day Cases Rate per 100000,7-Day Death Rate per 100000
1,Alaska,60333,0,0,950,8247,309,NaN,NaN,1,42,129.9,0.1
2,Alabama,515388,401490,113898,2677,10511,10554,8327,2227,67,215,54.6,1.4
3,Arkansas,330398,0,0,1221,10948,5626,NaN,NaN,66,186,40.5,2.2
4,American Samoa,0,0,0,0,0,0,NaN,NaN,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,Washington,364486,0,0,6987,4786,5247,NaN,NaN,47,68,91.8,0.6
58,Wisconsin,636041,577195,58846,4740,10924,7315,6622,693,52,125,81.4,0.9
59,West Virginia,141738,112315,29423,2920,7909,2676,NaN,NaN,57,149,162.9,3.2
60,Wyoming,56310,47654,8656,423,9729,700,700,0,5,120,73.1,0.9


In [5]:
df1=df = pd.read_csv("https://raw.githubusercontent.com/camckenzie/COVID-19-Interactive-Map/main/covid19_vaccinations.csv",skiprows=3,header = None)

In [6]:
df1

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,State/Territory/Federal Entity,Total Doses Delivered,Doses Delivered per 100K,18+ Doses Delivered per 100K,Total Doses Administered by State where Admini...,Doses Administered per 100k by State where Adm...,18+ Doses Administered by State where Administ...,18+ Doses Administered per 100K by State where...,People with at least One Dose by State of Resi...,Percent of Total Pop with at least One Dose by...,...,Percent of 65+ Pop with at least One Dose by S...,People 65+ Fully Vaccinated by State of Residence,Percent of 65+ Pop Fully Vaccinated by State o...,People 65+ Fully Vaccinated_Moderna_Resident,People 65+ Fully Vaccinated_Pfizer_Resident,People 65+ Fully Vaccinated_Janssen_Resident,People 65+ Fully Vaccinated_Unknown 2-dose Man...,65+ Doses Administered by State where Administ...,Doses Administered per 100k of 65+ pop by Stat...,Doses Delivered per 100k of 65+ pop
1,Alaska,615825,84181,111651,411529,56255,405536,73525,250378,34.2,...,72.5,55227,60.3,32313,22276,634,4,119391,130357,672386
2,Alabama,2839540,57912,74433,1771979,36139,1769781,46392,1184284,24.2,...,64,363605,42.8,189317,158576,15688,24,873963,102839,334128
3,Arkansas,1814280,60119,78281,1209965,40094,1206997,52079,803744,26.6,...,65.4,223201,42.6,125649,92667,4841,44,559566,106811,346315
4,American Samoa,43810,78669,100859,31167,55966,30221,69574,17856,32.1,...,23.5,1875,19.8,712,1162,1,0,4036,42632,462765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Vermont,432900,69376,84885,333676,53475,332898,65276,212105,34,...,87.8,65516,52.4,27886,37473,115,42,179583,143622,346212
60,Washington,4571720,60037,76812,3659886,48062,3649741,61321,2309959,30.3,...,79.1,735287,60.8,341632,367179,26105,371,1680069,138880,377915
61,Wisconsin,3399265,58382,74613,2969294,50997,2960067,64973,1890610,32.5,...,80.6,632649,62.2,304942,320570,7104,33,1471303,144636,334165
62,West Virginia,1152405,64303,80443,896388,50018,891868,62256,547917,30.6,...,66.5,190507,51.9,92768,95915,1810,14,436331,118888,313997


In [17]:
file_path = os.getcwd()

doc_path = file_path + "\\docs\\"

html_path = "https://github.com/camckenzie/COVID-19-Interactive-Map/tree/main/Docs/Cases_PNGs"

df["cases_file_location"] = html_path + "state_" + ".png"

html_path = "https://github.com/camckenzie/COVID-19-Interactive-Map/tree/main/Docs/Vaccinations_PNGs"

df["deaths_file_location"] = html_path + "state_" + ".png"

In [18]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,43,44,45,46,47,48,49,50,cases_file_location,deaths_file_location
0,State/Territory/Federal Entity,Total Doses Delivered,Doses Delivered per 100K,18+ Doses Delivered per 100K,Total Doses Administered by State where Admini...,Doses Administered per 100k by State where Adm...,18+ Doses Administered by State where Administ...,18+ Doses Administered per 100K by State where...,People with at least One Dose by State of Resi...,Percent of Total Pop with at least One Dose by...,...,Percent of 65+ Pop Fully Vaccinated by State o...,People 65+ Fully Vaccinated_Moderna_Resident,People 65+ Fully Vaccinated_Pfizer_Resident,People 65+ Fully Vaccinated_Janssen_Resident,People 65+ Fully Vaccinated_Unknown 2-dose Man...,65+ Doses Administered by State where Administ...,Doses Administered per 100k of 65+ pop by Stat...,Doses Delivered per 100k of 65+ pop,https://github.com/camckenzie/COVID-19-Interac...,https://github.com/camckenzie/COVID-19-Interac...
1,Alaska,615825,84181,111651,411529,56255,405536,73525,250378,34.2,...,60.3,32313,22276,634,4,119391,130357,672386,https://github.com/camckenzie/COVID-19-Interac...,https://github.com/camckenzie/COVID-19-Interac...
2,Alabama,2839540,57912,74433,1771979,36139,1769781,46392,1184284,24.2,...,42.8,189317,158576,15688,24,873963,102839,334128,https://github.com/camckenzie/COVID-19-Interac...,https://github.com/camckenzie/COVID-19-Interac...
3,Arkansas,1814280,60119,78281,1209965,40094,1206997,52079,803744,26.6,...,42.6,125649,92667,4841,44,559566,106811,346315,https://github.com/camckenzie/COVID-19-Interac...,https://github.com/camckenzie/COVID-19-Interac...
4,American Samoa,43810,78669,100859,31167,55966,30221,69574,17856,32.1,...,19.8,712,1162,1,0,4036,42632,462765,https://github.com/camckenzie/COVID-19-Interac...,https://github.com/camckenzie/COVID-19-Interac...


In [19]:
map_projection = "epsg:2163"

In [20]:
cwd = os.getcwd()

county_shape = cwd + "//shapefiles//lake//ne_10m_lakes.shx"

lake_map = gpd.read_file(county_shape)

lake_map = lake_map.to_crs({'init': map_projection})

In [21]:
cwd = os.getcwd()

county_shape = cwd + "//shapefiles//land//ne_50m_land.shx"

land_map = gpd.read_file(county_shape)

land_map = land_map.to_crs({'init': map_projection})

land_map = land_map.iloc[0:1200]

In [22]:
cwd = os.getcwd()

county_shape = cwd + "//shapefiles//county//tl_2017_us_county.shx"

us_map = gpd.read_file(county_shape)

us_map = us_map.to_crs({'init': map_projection})

In [23]:
us_map["geometry"] = us_map["geometry"].simplify(2000)

In [24]:
us_map["area_fips"] = (us_map.STATEFP.astype(str) + us_map.COUNTYFP.astype(str)).astype(int)

---

###  Step \#2 Merge and Clean

The next couple of cells download the requisite shapefiles from the US census. They are unzipped in a folder called shapefiles and then county. So they are assuming some structure behind your folder setup. 

In [25]:
us_map.head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,area_fips
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,None,None,None,A,1477641638,10701538,+41.9158651,-096.7885168,"POLYGON ((246290.041 -328619.667, 245963.100 -...",31039
1,53,069,01513275,53069,Wahkiakum,Wahkiakum County,06,H1,G4020,None,None,None,A,680956787,61588406,+46.2946377,-123.4244583,"POLYGON ((-1770686.984 398793.664, -1772831.10...",53069
2,35,011,00933054,35011,De Baca,De Baca County,06,H1,G4020,None,None,None,A,6016761648,29147345,+34.3592729,-104.3686961,"POLYGON ((-422754.447 -1210188.449, -442958.76...",35011
3,31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,30700,None,A,2169252486,22867561,+40.7835474,-096.6886584,"POLYGON ((259511.654 -444450.123, 259130.939 -...",31109
4,31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,None,None,None,A,1489645186,1718484,+40.1764918,-098.0468422,"POLYGON ((146978.459 -544334.301, 146407.623 -...",31129
